In [ ]:
'''
В модели генитор (Genitor) используется специфичный способ отбора.
Вначале, как и полагается, популяция инициализируется, и ее особи оцениваются.
Затем выбираются случайным образом две особи, скрещиваются, причем получается только один потомок,
который оценивается и занимает место менее приспособленной особи в популяции (а не одного из родителей!).
После этого снова случайным образом выбираются две особи, и их потомок занимает место
родительской особи с самой низкой приспособленностью. Таким образом, на каждом шаге в популяции
обновляется лишь одна особь. Процесс продолжается до тех пор, пока пригодности хромосом не станут одинаковыми. 
В данный алгоритм можно добавить мутацию потомка после его создания. 
Критерий окончания процесса, как и вид кроссинговера и мутации, можно выбирать разными способами.''';

In [ ]:
# МЫ МАКСИМИЗИРУЕМ

In [1]:
from random import sample
import numpy as np

In [192]:
np.log2(10**PRECISION * 15)

30.482315354707417

In [194]:
# num of symbols after point
PRECISION = 8
BITS = 40
BITS_CODE = '0{0}b'.format(BITS)

In [195]:
def square_func(vector):
    return -np.sum(vector ** 2)

def rastrigin_func(vector):
    return -np.sum(vector**2 - 10 * np.cos(2*np.pi*vector))

def eggholder(vector):
    assert len(vector) == 2, 'Not appropriate dim, use 2'
    x, y = vector[0], vector[1]
    if abs(x) > 512. or abs(y) > 512.:
        return -1000
    return (y+47)*np.sin(abs(x*0.5 + y + 47)**0.5) + x*np.sin(abs(x-y-47)**0.5)

def bukin(vector):
    assert len(vector) == 2, 'Not appropriate dim, use 2'
    x, y = vector[0], vector[1]
    if x < -15 or x > -5 or abs(y) > 3:
        return -100.
    return -(100 * (abs(y - 0.01*x*x)**0.5) + 0.01*abs(x + 10))

In [223]:
FUNCTION = bukin#eggholder#rastrigin_func
MAX_ITER = 10000
RANGE = 15.
BITS = int(np.log2(10**PRECISION * 15)) + 1

In [227]:
POPULATION_LENGTH = 100
population = generate_population(POPULATION_LENGTH, 2, range_=RANGE)
#print(list(map(lambda x:x.features, population)))

for iter_num in range(MAX_ITER):
    sp1, sp2 = sample(population, 2)
    child = sp1.cross_over(sp2).mutate(iter_num, 1)
    population.sort(key=lambda x: -x.assess(FUNCTION))
    if population[POPULATION_LENGTH - 1].assess(FUNCTION) < child.assess(FUNCTION):
        population[POPULATION_LENGTH - 1] = child

    idsp1, idsp2 = sample(range(0, POPULATION_LENGTH), 2)
    sp1 = population[idsp1]
    sp2 = population[idsp2]
    child = sp1.cross_over(sp2).mutate(iter_num, 1)
#     if child.assess(FUNCTION) < sp1.assess(FUNCTION) and child.assess(FUNCTION) < sp2.assess(FUNCTION):
#         continue
    if sp1.assess(FUNCTION) < sp2.assess(FUNCTION):
        population[idsp1] = child
    else:
        population[idsp2] = child
        
population.sort(key=lambda x: -x.assess(FUNCTION))
#print(population[0].features)
print(list(map(lambda x:x.features, population)))
print(list(map(lambda x:x.assess(FUNCTION), population)))

[array([-9.93596453,  0.98724471]), array([-9.93594402,  0.9872447 ]), array([-9.93600546,  0.98722422]), array([-9.93756198,  0.98760312]), array([-9.94027556,  0.98814584]), array([-9.93918499,  0.98781815]), array([-9.93877028,  0.98768503]), array([-9.93932324,  0.98771575]), array([-9.93728036,  0.98728567]), array([-9.93909795,  0.98761335]), array([-9.93625126,  0.98754168]), array([-9.93436706,  0.98726518]), array([-9.98054948,  0.9957439 ]), array([-9.93604644,  0.98772599]), array([-9.93859109,  0.98726519]), array([-9.93882146,  0.98722422]), array([-10.02307108,   1.00350584]), array([-9.92810532,  0.98729591]), array([-9.92918563,  0.98770551]), array([-9.75910434,  0.95016567]), array([-9.92844835,  0.98843255]), array([-9.74090787,  0.95155831]), array([-9.93597476,  0.9905215 ]), array([-9.59919136,  0.91724407]), array([-9.59918112,  0.91720311]), array([-9.93660965,  0.99283575]), array([-9.97077025,  0.98749047]), array([-9.93637922,  0.97980023]), array([-9.9811280

In [208]:
def generate_population(num_species, n_dim, range_= 50.):
    population = []
    for _ in range(num_species):
        population.append(Species(n_dim, range_))
    return population

In [6]:
def invert(bit):
    return '0' if bit == '1' else '1'

In [7]:
def generate_vector(n_dim, range_):
    return np.clip(np.random.randn(n_dim) * range_ * 0.5, -range_, range_)

In [8]:
def decimal2binary(vector):
    return list(map(lambda x: format(int(x * 10 ** PRECISION), BITS_CODE), list(vector)))

In [9]:
def binary2decimal(vector):
    return np.array(list(map(lambda x: int(x, 2) / (10 ** PRECISION), vector)))

In [167]:
class Species:
    def __init__(self, n_dim=1, range_=50., features=[]):
        if len(features) == 0:
            self.features = generate_vector(n_dim, range_)
        else:
            self.features = features
    
    def mutate(self, decay=0, rate=1):
        genetic_code = decimal2binary(self.features)
        min_boundary = 0#BITS-35# min(BITS-PRECISION, round((BITS-1)*decay/MAX_ITER))
        for _ in range(rate):
            for i, feature in enumerate(genetic_code):
                chromosome = np.random.randint(min_boundary, BITS-1)
                if feature[0] == '-':
                    chromosome += 1
                bit = feature[chromosome]
                mutated = feature[:chromosome] + invert(bit) + feature[chromosome + 1:]
                genetic_code[i] = mutated
            self.features = binary2decimal(genetic_code)
        return self
        
    def cross_over(self, partner):
        gens1 = decimal2binary(self.features)
        gens2 = decimal2binary(partner.features)
        child_gens = []
        for i, feature in enumerate(gens1):
            chromosome_start = np.random.randint(0, BITS - 1)
            chromosome_end = np.random.randint(chromosome_start + 1, BITS)
            
            if feature[0] == '-':
                chromosome_start += 1
                chromosome_end += 1
            
            child_gens.append(feature[:chromosome_start] + gens2[i][chromosome_start:chromosome_end] 
                              + feature[chromosome_end:])
        return Species(features = binary2decimal(child_gens))
    
    def assess(self, function):
        return function(self.features)